In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
from models import *
from configs import cfg
import pandas as pd
from nltk.translate import bleu_score
import pickle

In [2]:
def load_data(fname):
    # From the csv file given by filename and return a pandas DataFrame of the read csv.
    
    # Define path to data
    # Note: Path relative to KHIEM's files, make changes to relative path if necessary
    dataPath = 'BeerAdvocatePA4/' + fname
    
    # Read csv into pandas frame
    df = pd.read_csv(dataPath)
    
    # Return frame
    return df

def char2oh(padded,translate, beers):
    # Each row has form: beer style index||overall||character indices
    # TODO: Onehot the beer style and concatenate to overall
    # Onehot the character indices and concatenate the beerstyle||overall to each onehotted char
    
    # Each row's beer has form 1x#ofpossiblebeers
    beerstyles = np.zeros((padded.shape[0],len(beers)))
    # Each row's review has form max(lenofsequence) x #ofpossiblecharacters
    # Since we padded each review, they all have the same number of characters
    # Subtract two since we know the first two values aren't characters
    chars = np.zeros((padded.shape[0], (padded.shape[1] - 2), len(translate)))
    
    # First two columns are beerstyle indices and overalls
    bsidx = padded[:,0]
    ovrl = padded[:,1]
    
    # The rest are characters
    ch = padded[:,2:]
    
    # Index with bsidx
    beerstyles[np.arange(padded.shape[0]), bsidx.astype(int)] = 1
    
    igrid = np.mgrid[0:padded.shape[0], 0:(padded.shape[1]-2)]
    # Index with ch, we use meshgrid since this is a 3d array
    chars[igrid[0], igrid[1], ch.astype(int)] = 1
    
    # Concatenate overall and beer style
    meta_data = np.c_[ovrl, beerstyles]
    
    # Tile and reshape meta_data so we have a copy for each one of the characters
    tiled_meta = np.tile(meta_data, padded.shape[1] - 2).reshape(padded.shape[0], (padded.shape[1] - 2), -1)
    
    # Concatenate the items 
    
    # Return both the concatenated and just the one hot
    return np.c_[tiled_meta, chars],chars
    

def process_train_data(data):
    # TODO: Input is a pandas DataFrame and return a numpy array (or a torch Tensor/ Variable)
    # that has all features (including characters in one hot encoded form).

    # Get the dictionary to translate between ASCII and onehot index
    with open("ASCII2oneHot.pkl", "rb") as f:
        translate = pickle.load(f)
    
    # Get the dictionary to translate between beer style and index
    with open("BeerDict.pkl", "rb") as f:
        beers = pickle.load(f)
        
    # List of reviews to onehot after translation
    toOnehot = []
    
    # For each review, convert to list of its translated characters
    # Translated means ord(c) -> onehot index
    # Also translate the beer style to its index value
    # Concatenate all the data and convert to tensor
    for idx,rev in data.iterrows():
        if isinstance(rev['review/text'], str):
            toOnehot.append(torch.Tensor([beers[rev['beer/style']]] + [rev['review/overall']] + 
                                         [translate[ord(x)] for x in list(chr(0)+rev['review/text']+chr(1))]))
    
    # Pad all smaller sentences with 1s to signify <EOS>
    padded = pad_data(toOnehot, translate[1])
    del toOnehot

    # Take the array padded sentences and one-hot the characters.
    # Beer style also gets one-hot
    # Overall does not
    reviews,labels = char2oh(np.array(padded), translate, beers)
    del padded
    
    # Since the labels are simply the next characters, we take all characters except the last one
    # for the review, and everything but the first one for the labels
    # Argmax to get back the character indices for the labels
    return torch.Tensor(reviews[:,0:-1,:]), torch.Tensor(labels[:,1:,:]).argmax(dim=2)
    
    
def train_valid_split(data):
    # TODO: Takes in train data as dataframe and
    # splits it into training and validation data.
    
    # List of indices of the data
    ind = np.arange(len(data))
    
    # Randomize the split
    np.random.shuffle(ind)
    
    # Where to split the indices
    # We'll take first 20% for validation, the rest for training
    split = int(0.2*len(data))
    
    # Split the indices
    vIndices = ind[0:split]
    tIndices = ind[split:]
    
    # Group the indices into their frames then return those
    validation_data = data.iloc[vIndices]
    train_data = data.iloc[tIndices]
    
    return train_data,validation_data
    
def process_test_data(data):
    # TODO: Takes in pandas DataFrame and returns a numpy array (or a torch Tensor/ Variable)
    # that has all input features. Note that test data does not contain any review so you don't
    # have to worry about one hot encoding the data.
    raise NotImplementedError

    
def pad_data(orig_data, pad):
    # TODO: Since you will be training in batches and training sample of each batch may have reviews
    # of varying lengths, you will need to pad your data so that all samples have reviews of length
    # equal to the longest review in a batch. You will pad all the sequences with <EOS> character 
    # representation in one hot encoding.
    # Data comes in as translated ASCII representation, simply sort and call torch pad
    return torch.nn.utils.rnn.pad_sequence(sorted(orig_data, key = lambda x: len(x), reverse=True), 
                                           batch_first=True, padding_value=pad)
    

def getBatchIter(data, batchSize):
    # TODO: Returns a list of batches of indices
    # The list of batch indices will be used to index into the
    # corresponding data frame to extract the data
    
    # List of all possible indices
    ind = np.arange(len(data))
    
    # Calculate how many batches of batchSize would fit into
    # into the length of the data
    numBatches = int(len(data)/batchSize)
    
    # Split the array of indices into roughly equivalent batch sized batches
    batchedInd = np.array_split(ind, numBatches)
    
    return batchedInd
    
    
def train(model, X_train, X_valid, cfg):
    # TODO: Train the model!
    # Datas are given as pandas data frame. Call process on-line as we train to
    # get the data and label
    
    epochs = np.arange(1)
    l_rate = cfg['learning_rate']
    penalty = cfg['L2_penalty']
    
    
    # Define loss and optimizer
    Criterion = torch.nn.NLLLoss() # We'll use NLL
    Optimizer = optim.Adam(model.parameters(), lr=l_rate, weight_decay=penalty) # Let's use ADAM
    
    # Size of each batch
    batchSize = 150
    
    # Create the batch iterator for the data
    trainIter = getBatchIter(X_train, batchSize)
    validIter = getBatchIter(X_valid, batchSize)
    
    all_loss = []
    for e in epochs:
        batch_loss = 0
        # Training loop
        for batch_count, batchInd in enumerate(trainIter,0):
            # Get the dataframe for the batch
            batchFrame = X_train.iloc[batchInd]

            # Process the batch for data and labels
            batch, labels = process_train_data(batchFrame)
            batch,labels = batch.to(computing_device), labels.to(computing_device)

            # TODO: Actual fuckin train lmao

            # Run our batch through the model
            # batch has shape Batchsize x Seqlen x Input Dim
            output, (h,c) = model(batch)
            
            # Save space
            del h
            del c
            
            # Reshape the output and labels to so that the loss function
            # can simply interpret time as another batch
            # This will be fine since sum of sum can be thought of as just
            # one sum
            output = output.view(output.shape[0]*output.shape[1], output.shape[2])
            labels = labels.view(-1)

            # Get loss and compute gradients
            loss = Criterion(output,labels)
            loss.backward()
            del output

            batch_loss += float(loss)
            del loss
            
            # Optimize step
            Optimizer.step()

            # Progress bar
            if batch_count % 50 == 0 and batch_count > 0:
                batch_loss /= 50
                print("On batch %d with loss %f" % (batch_count, batch_loss))
                all_loss.append(batch_loss)
                
            
        print("Completed epoch %d" % e)
    
    print("Completed Training")
        
    
    
def generate(model, X_test, cfg):
    # TODO: Given n rows in test data, generate a list of n strings, where each string is the review
    # corresponding to each input row in test data.
    raise NotImplementedError
    
    
def save_to_file(outputs, fname):
    # TODO: Given the list of generated review outputs and output file name, save all these reviews to
    # the file in .txt format.
    raise NotImplementedError
    

In [3]:
if __name__ == "__main__":
    train_data_fname = "Beeradvocate_Train.csv"
    test_data_fname = "Beeradvocate_Test.csv"
    out_fname = "model_outputs.out"
    
    train_data = load_data(train_data_fname) # Generating the pandas DataFrame
    test_data = load_data(test_data_fname) # Generating the pandas DataFrame
    X_train, X_valid = train_valid_split(train_data) # Splitting the train data into train-valid data
    
    model = baselineLSTM(cfg) # Replace this with model = <your model name>(cfg)
    if cfg['cuda']:
        computing_device = torch.device("cuda")
    else:
        computing_device = torch.device("cpu")
    model.to(computing_device)
    
    train(model, X_train, X_valid, cfg) # Train the model
    outputs = generate(model, X_test, cfg) # Generate the outputs for test data
    save_to_file(outputs, out_fname) # Save the generated outputs to a file

On batch 1 with loss 0.188830
On batch 2 with loss 0.096572
On batch 3 with loss 0.092879
On batch 4 with loss 0.091354
On batch 5 with loss 0.091814
On batch 6 with loss 0.090613
On batch 7 with loss 0.092001
On batch 8 with loss 0.091314
On batch 9 with loss 0.090576
On batch 10 with loss 0.091126
On batch 11 with loss 0.090969
On batch 12 with loss 0.091668
On batch 13 with loss 0.092555
On batch 14 with loss 0.090852
On batch 15 with loss 0.091234
On batch 16 with loss 0.091067
On batch 17 with loss 0.091175
On batch 18 with loss 0.091651
On batch 19 with loss 0.091109
On batch 20 with loss 0.090930
On batch 21 with loss 0.091982
On batch 22 with loss 0.090757
On batch 23 with loss 0.090829
On batch 24 with loss 0.091735
On batch 25 with loss 0.090333
On batch 26 with loss 0.090888
On batch 27 with loss 0.090429
On batch 28 with loss 0.091963
On batch 29 with loss 0.090941
On batch 30 with loss 0.090744
On batch 31 with loss 0.091633
On batch 32 with loss 0.090942
On batch 33 with 

KeyboardInterrupt: 